In [240]:
import numpy as np
import pandas as pd

In [241]:
# --- Data Loading ---
def load_yield_data(filepath, target_date):
    """Load and process yield curve data for a specific date."""
    try:
        df = pd.read_csv(filepath)
        data_on_date = df[df['Date'] == target_date].copy()
        if data_on_date.empty:
            raise ValueError(f"No data found for date {target_date}")
        
        maturities = sorted(set(col.split('_')[1] for col in df.columns if 'Time_' in col))
        
        for mat in maturities:
            time_col = f'Time_{mat}'
            yield_col = f'Yield_{mat}'
            if time_col in data_on_date.columns:
                data_on_date.loc[:, time_col] = pd.to_numeric(data_on_date[time_col], errors='coerce')
            if yield_col in data_on_date.columns:
                data_on_date.loc[:, yield_col] = pd.to_numeric(data_on_date[yield_col], errors='coerce')
        
        valid_cols = [f'Time_{mat}' for mat in maturities if f'Time_{mat}' in data_on_date.columns] + \
                     [f'Yield_{mat}' for mat in maturities if f'Yield_{mat}' in data_on_date.columns]
        data_on_date = data_on_date.dropna(subset=valid_cols)
        
        times = np.array([data_on_date[f'Time_{mat}'].values[0] for mat in maturities])
        yields = np.array([data_on_date[f'Yield_{mat}'].values[0] for mat in maturities]) * 100
        valid_idx = ~np.isnan(times) & ~np.isnan(yields)
        return times[valid_idx], yields[valid_idx]
    except Exception as e:
        raise Exception(f"Error loading data for {target_date}: {str(e)}")


In [242]:
# # Load Jan 31st data
# times_jan31, yields_jan31 = load_yield_data(
#     '/Users/dr/Documents/GitHub/FixedIncome/STRIPS_data.csv', 
#     '2025-01-31'
# )

In [243]:
# # Load STRIPS data
# df = pd.read_csv('/Users/dr/Documents/GitHub/FixedIncome/STRIPS_data.csv')
# dt = '2025-01-31'  # Changed to January 31, 2025
# data_on_date = df[df['Date'] == dt]
# maturities = sorted(set(col.split('_')[1] for col in df.columns if 'Time_' in col))
# times_jan31 = np.array([data_on_date[f'Time_{mat}'].values[0] for mat in maturities])
# yields_jan31 = np.array([data_on_date[f'Yield_{mat}'].values[0] for mat in maturities]) * 100  # Convert to percentage
# valid_idx = ~np.isnan(times_jan31) & ~np.isnan(yields_jan31)
# times_jan31 = times_jan31[valid_idx]
# yields_jan31 = yields_jan31[valid_idx]

In [244]:
# yields_jan31

In [245]:
# Two-Factor Vasicek Model (with adjusted yield curve formula)
# class Two_Factor_Vasicek:
#     def __init__(self, x1_0, x2_0, mu1, mu2, vol1, vol2, kappa1, kappa2):
#         self.x1_0 = x1_0
#         self.x2_0 = x2_0
#         self.mu1 = mu1
#         self.mu2 = mu2
#         self.vol1 = vol1
#         self.vol2 = vol2
#         self.var1 = vol1**2
#         self.var2 = vol2**2
#         self.kappa1 = kappa1
#         self.kappa2 = kappa2
    
#     def B(self, tau, kappa):
#         return (1 - np.exp(-kappa * tau)) / kappa if kappa > 1e-6 else tau  # Avoid division by zero
    
#     def A(self, tau, mu, var, kappa):
#         B_tau = self.B(tau, kappa)
#         return (mu - var / (2 * kappa**2)) * (B_tau - tau) - (var * B_tau**2) / (4 * kappa)
    
#     def yield_curve(self, tau, r_t):
#         if tau <= 0:
#             return np.nan
#         A1 = self.A(tau, self.mu1, self.var1, self.kappa1)
#         A2 = self.A(tau, self.mu2, self.var2, self.kappa2)
#         B1 = self.B(tau, self.kappa1)
#         B2 = self.B(tau, self.kappa2)
#         price = np.exp(A1 + A2 - (B1 + B2) * r_t)
#         price = max(price, 1e-10)
#         return -np.log(price) / tau
    
#     def update_params(self, x):
#         self.x1_0 = max(x[0], 0)
#         self.x2_0 = max(x[1], 0)
#         self.mu1 = x[2]
#         self.mu2 = x[3]
#         self.vol1 = max(x[4], 1e-6)
#         self.vol2 = max(x[5], 1e-6)
#         self.var1 = self.vol1**2
#         self.var2 = self.vol2**2
#         self.kappa1 = max(x[6], 1e-6)
#         self.kappa2 = max(x[7], 1e-6)

#     def simulate_paths(self, T, dt, n_paths):
#         n_steps = int(T / dt)
#         times = np.linspace(0, T, n_steps + 1)
#         r_paths = np.zeros((n_paths, n_steps + 1))
#         x1 = np.zeros((n_paths, n_steps + 1))
#         x2 = np.zeros((n_paths, n_steps + 1))
#         x1[:, 0] = self.x1_0
#         x2[:, 0] = self.x2_0
#         r_paths[:, 0] = x1[:, 0] + x2[:, 0]

#         for t in range(1, n_steps + 1):
#             dW1 = np.random.normal(0, np.sqrt(dt), n_paths)
#             dW2 = np.random.normal(0, np.sqrt(dt), n_paths)
#             x1[:, t] = x1[:, t-1] + self.kappa1 * (self.mu1 - x1[:, t-1]) * dt + self.vol1 * dW1
#             x2[:, t] = x2[:, t-1] + self.kappa2 * (self.mu2 - x2[:, t-1]) * dt + self.vol2 * dW2
#             r_paths[:, t] = np.maximum(x1[:, t] + x2[:, t], 0)

#         return times, r_paths

In [246]:
# --- Two-Factor Vasicek Model ---
class Two_Factor_Vasicek:
    def __init__(self, x1_0, x2_0, mu1, mu2, vol1, vol2, kappa1, kappa2):
        self.update_params([x1_0, x2_0, mu1, mu2, vol1, vol2, kappa1, kappa2])
    
    def B(self, tau, kappa):
        return (1 - np.exp(-kappa * tau)) / kappa if kappa > 1e-6 else tau
    
    def A(self, tau, mu, var, kappa):
        B_tau = self.B(tau, kappa)
        return (mu - var / (2 * kappa**2)) * (B_tau - tau) - (var * B_tau**2) / (4 * kappa)
    
    def yield_curve(self, tau, r_t):
        if tau <= 0:
            return np.nan
        A1 = self.A(tau, self.mu1, self.var1, self.kappa1)
        A2 = self.A(tau, self.mu2, self.var2, self.kappa2)
        B1 = self.B(tau, self.kappa1)
        B2 = self.B(tau, self.kappa2)
        price = np.exp(A1 + A2 - (B1 + B2) * r_t)
        price = max(price, 1e-10)
        return -np.log(price) / tau
    
    def update_params(self, x):
        self.x1_0 = max(x[0], 0)
        self.x2_0 = max(x[1], 0)
        self.mu1 = x[2]
        self.mu2 = x[3]
        self.vol1 = max(x[4], 1e-6)
        self.vol2 = max(x[5], 1e-6)
        self.var1 = self.vol1**2
        self.var2 = self.vol2**2
        self.kappa1 = max(x[6], 1e-6)
        self.kappa2 = max(x[7], 1e-6)
    
    def simulate_paths(self, T, dt, n_paths):
        n_steps = int(T / dt)
        times = np.linspace(0, T, n_steps + 1)
        r_paths = np.zeros((n_paths, n_steps + 1))
        x1 = np.zeros((n_paths, n_steps + 1))
        x2 = np.zeros((n_paths, n_steps + 1))
        x1[:, 0] = self.x1_0
        x2[:, 0] = self.x2_0
        r_paths[:, 0] = x1[:, 0] + x2[:, 0]
        
        for t in range(1, n_steps + 1):
            dW1 = np.random.normal(0, np.sqrt(dt), n_paths)
            dW2 = np.random.normal(0, np.sqrt(dt), n_paths)
            x1[:, t] = x1[:, t-1] + self.kappa1 * (self.mu1 - x1[:, t-1]) * dt + self.vol1 * dW1
            x2[:, t] = x2[:, t-1] + self.kappa2 * (self.mu2 - x2[:, t-1]) * dt + self.vol2 * dW2
            r_paths[:, t] = np.maximum(x1[:, t] + x2[:, t], 0)
        
        return times, r_paths

In [247]:
from scipy.optimize import least_squares

In [248]:
# Fitter Class
# class Fitter:
#     def __init__(self, model, obs_yields, obs_times, dist='Q'):
#         self.model = model
#         self.data = np.array(obs_yields)  # Observed yields in decimal
#         self.times = np.array(obs_times)
#         self.dist = dist
#         self.model_yields = None
#         self.n_params = 8

#     def residuals(self, x):
#         m = self.model
#         m.update_params(x)
#         r0 = m.x1_0 + m.x2_0
        
#         # Calculate model yields
#         model_yields = np.array([m.yield_curve(t, r0) for t in self.times])
#         residuals = model_yields - self.data
        
#         # Use time-weighted scheme - emphasize short-term yields more
#         weights = 1 / (1 + self.times)  # Higher weight for shorter maturities
        
#         # Additional weight for the first 3 maturities
#         weights[:3] *= 2
        
#         weighted_residuals = residuals * weights
        
#         # Improved penalty terms
#         short_term_penalty = 1000 * (model_yields[0] - self.data[0])**2  # Exact match for shortest maturity
#         level_penalty = 100 * (np.mean(model_yields) - np.mean(self.data))**2  # Match average level
#         convexity_penalty = 50 * ((model_yields[-1] - model_yields[0]) - (self.data[-1] - self.data[0]))**2  # Match curve steepness
        
#         # Regularization to prevent extreme parameters
#         reg_penalty = 0.01 * np.sum(x**2)
        
#         total_penalty = short_term_penalty + level_penalty + convexity_penalty + reg_penalty
        
#         return np.append(weighted_residuals, total_penalty)
    
#     def r_squared(self):
#         if self.model_yields is None:
#             raise ValueError("Model must be fitted first.")
#         y_mean = np.mean(self.data)
#         ss_tot = np.sum((self.data - y_mean) ** 2)
#         ss_res = np.sum((self.data - self.model_yields) ** 2)
#         return 1 - (ss_res / ss_tot) if ss_tot > 0 else np.nan
    
#     def rmse(self):
#         if self.model_yields is None:
#             raise ValueError("Model must be fitted first.")
#         return np.sqrt(np.mean((self.data - self.model_yields) ** 2))

#     def fit(self, x0, solver='trf', bounds=None):
#         if bounds is not None:
#             lower_bounds, upper_bounds = zip(*bounds)
#             bounds_for_least_squares = (list(lower_bounds), list(upper_bounds))
#         else:
#             bounds_for_least_squares = ([0, 0, -np.inf, -np.inf, 0, 0, 0, 0], 
#                                        [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf])

#         result = least_squares(self.residuals, x0, method=solver, bounds=bounds_for_least_squares)
#         self.model.update_params(result.x)
#         self.model_yields = np.array([self.model.yield_curve(t, self.model.x1_0 + self.model.x2_0) for t in self.times])
#         return result

In [249]:
# --- Fitter Class ---
class Fitter:
    def __init__(self, model, obs_yields, obs_times, dist='Q'):
        self.model = model
        self.data = np.array(obs_yields)
        self.times = np.array(obs_times)
        self.dist = dist
        self.model_yields = None
    
    def residuals(self, x):
        self.model.update_params(x)
        r0 = self.model.x1_0 + self.model.x2_0
        model_yields = np.array([self.model.yield_curve(t, r0) for t in self.times])
        residuals = model_yields - self.data
        weights = 1 / (1 + self.times)
        weights[:3] *= 2
        weighted_residuals = residuals * weights
        
        short_term_penalty = 1000 * (model_yields[0] - self.data[0])**2
        level_penalty = 100 * (np.mean(model_yields) - np.mean(self.data))**2
        convexity_penalty = 50 * ((model_yields[-1] - model_yields[0]) - (self.data[-1] - self.data[0]))**2
        reg_penalty = 0.01 * np.sum(x**2)
        total_penalty = short_term_penalty + level_penalty + convexity_penalty + reg_penalty
        
        return np.append(weighted_residuals, total_penalty)
    
    def r_squared(self):
        if self.model_yields is None:
            raise ValueError("Model must be fitted first.")
        y_mean = np.mean(self.data)
        ss_tot = np.sum((self.data - y_mean) ** 2)
        ss_res = np.sum((self.data - self.model_yields) ** 2)
        return 1 - (ss_res / ss_tot) if ss_tot > 0 else np.nan
    
    def rmse(self):
        if self.model_yields is None:
            raise ValueError("Model must be fitted first.")
        return np.sqrt(np.mean((self.data - self.model_yields) ** 2))
    
    def fit(self, x0, solver='trf', bounds=None):
        bounds_for_least_squares = bounds if bounds else (
            [0, 0, -np.inf, -np.inf, 0, 0, 0, 0],
            [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf]
        )
        result = least_squares(self.residuals, x0, method=solver, bounds=bounds_for_least_squares)
        self.model.update_params(result.x)
        self.model_yields = np.array([self.model.yield_curve(t, self.model.x1_0 + self.model.x2_0) for t in self.times])
        return result

In [250]:
# # Fit the model with adjusted parameters and bounds
# model = Two_Factor_Vasicek(0.01847, 0.01847, 0.0225, 0.0225, 0.005, 0.005, 0.05, 0.05)
# yields_jan31_decimal = yields_jan31 / 100  # Convert to decimal for fitting
# fitter = Fitter(model, yields_jan31_decimal, times_jan31)
# x0 = [
#     0.02,       # x1_0
#     0.02,       # x2_0
#     0.025,      # mu1
#     0.025,      # mu2
#     0.01,       # vol1
#     0.01,       # vol2
#     0.5,        # kappa1 (faster mean reversion)
#     0.1         # kappa2 (slower mean reversion)
# ]

# bounds = [
#         (0.01, 0.03),    # x1_0 (short rate component)
#         (0.01, 0.03),    # x2_0 (long rate component)
#         (0.015, 0.04),   # mu1 (long-term mean 1)
#         (0.015, 0.04),   # mu2 (long-term mean 2)
#         (0.001, 0.02),   # vol1 (volatility 1)
#         (0.001, 0.02),   # vol2 (volatility 2)
#         (0.05, 1.0),     # kappa1 (mean reversion 1) - faster for short rate
#         (0.01, 0.3)      # kappa2 (mean reversion 2) - slower for long rate
# ]

# fit_result = fitter.fit(x0, solver='trf', bounds=bounds)
# model.update_params(fit_result.x)
# print("Calibrated parameters [x1_0, x2_0, mu1, mu2, vol1, vol2, kappa1, kappa2]:", fit_result.x)

In [251]:
# Check the fitted yield curve at t=0
fitted_yields_jan31 = np.array([model.yield_curve(t, model.x1_0 + model.x2_0) * 100 for t in times_jan31])
print("Fitted yields at t=0 (Jan 31, 2025):", fitted_yields_jan31)
print("Observed yields (Jan 31, 2025):", yields_jan31)

Fitted yields at t=0 (Jan 31, 2025): [7.34936513 7.27803402 7.20757453 7.13786929 7.06936036 7.00190734
 6.93555293 6.87015497 6.80609882 6.74322588 6.44685373 7.38503553
 7.3133024  7.24240101 7.17240881 7.10320429 6.96831776 6.90253088
 6.83771697 6.77425494 6.65092028 6.59090757 6.53227926 6.4748705
 7.36747239 7.29593428 7.22524789 7.15529803 7.0865307  7.01880718
 6.95217227 6.8864849  6.8221337  6.7589608  6.57649946 6.51816857
 6.46105693 7.33130799 7.26018919 7.18996116 7.12050442 6.98507733
 6.91900564 6.85389887 6.79013913 6.72756717]
Observed yields (Jan 31, 2025): [3.694 4.066 4.158 4.157 4.294 4.368 4.425 4.457 4.493 4.525 4.701 4.159
 4.1   3.948 4.246 4.176 4.379 4.444 4.477 4.515 4.402 4.489 4.589 4.639
 4.221 4.208 4.214 4.258 4.304 4.332 4.409 4.447 4.487 4.517 4.524 4.582
 4.688 4.22  3.867 4.172 4.278 4.363 4.44  4.442 4.493 4.531]


In [252]:
import matplotlib.pyplot as plt

In [253]:
# Get metrics from the fitter
# r2 = fitter.r_squared()
# rmse = fitter.rmse()

# # Create formatted strings for legend
# metrics_text = f'$R^2$: {r2:.2f}\nRMSE: {rmse*100:.2f}%'

# print(metrics_text)

In [254]:
# def plot_yield_comparison(times, yields=None, fitted=None, forecast=None, historical=None,
#                          metrics_text=None, title=None, figsize=(16, 10), dpi=100):
#     """
#     Unified function to plot yield curve comparisons for both fitting and forecasting scenarios.
    
#     Parameters:
#     - times: Array of maturity times
#     - yields: Array of observed yields (optional)
#     - fitted: Array of fitted yields (for fitting plots)
#     - forecast: Dict with forecast stats (for forecasting plots) OR array of mean forecast values
#     - historical: List of dicts with historical series to plot
#     - metrics_text: String containing metrics to display
#     - title: Plot title
#     - figsize: Figure dimensions
#     - dpi: Figure resolution
#     """
#     # Sort all data by maturity
#     sorted_idx = np.argsort(times)
#     times_sorted = times[sorted_idx]
    
#     plt.figure(figsize=figsize, dpi=dpi)
    
#     # Plot observed yields if provided
#     if yields is not None:
#         yields_sorted = yields[sorted_idx]
#         obs_plot = plt.plot(times_sorted, yields_sorted, 'o-',
#                            label='Observed Yields',
#                            color='#1f77b4', linewidth=2,
#                            markersize=8, markerfacecolor='white',
#                            markeredgewidth=2)
    
#     # Plot fitted yields if provided (fitting scenario)
#     if fitted is not None:
#         fitted_sorted = fitted[sorted_idx]
#         fit_plot = plt.plot(times_sorted, fitted_sorted, 's--',
#                            label='Fitted Yields',
#                            color='#d62728', linewidth=2,
#                            markersize=6, alpha=0.9)
    
#     # Plot forecast if provided (forecasting scenario)
#     if forecast is not None:
#         # Handle both dict and array inputs for forecast
#         if isinstance(forecast, dict):
#             fcst_mean = forecast['mean']
#             fcst_lower = forecast.get('lower', None)
#             fcst_upper = forecast.get('upper', None)
#         else:
#             fcst_mean = forecast
#             fcst_lower = None
#             fcst_upper = None
        
#         fcst_mean_sorted = fcst_mean[sorted_idx]
#         fcst_plot = plt.plot(times_sorted, fcst_mean_sorted, 'D-',
#                             label='Forecasted Yields',
#                             color='#2ca02c', linewidth=2.5,
#                             markersize=7, alpha=0.9)
        
#         # Confidence band if available
#         if fcst_lower is not None and fcst_upper is not None:
#             plt.fill_between(times_sorted,
#                             fcst_lower[sorted_idx],
#                             fcst_upper[sorted_idx],
#                             color='#2ca02c', alpha=0.15,
#                             label=f"{forecast["confidence"]*100} % Confidence Band")
    
#     # Plot historical series if provided
#     if historical is not None:
#         for series in historical:
#             hist_times = series['times']
#             hist_yields = series['yields']
#             hist_sorted_idx = np.argsort(hist_times)
#             plt.plot(hist_times[hist_sorted_idx], hist_yields[hist_sorted_idx],
#                     series.get('marker', 'o'),
#                     label=series.get('label', 'Historical'),
#                     color=series.get('color', '#7f7f7f'),
#                     markersize=8, linestyle='none')
    
#     # Create legends
#     handles, labels = plt.gca().get_legend_handles_labels()
    
#     # Main legend for yield curves
#     if handles:  # Only add legend if there are items to show
#         main_legend = plt.legend(handles=handles, loc='upper right',
#                                fontsize=12, framealpha=1)
#         plt.gca().add_artist(main_legend)
    
#     # Add metrics text if provided
#     if metrics_text is not None:
#         metrics_patch = plt.Line2D([0], [0], marker='', color='none',
#                                  label=metrics_text,
#                                  markerfacecolor='white', markersize=0)
#         plt.legend(handles=[metrics_patch], loc='lower left',
#                  fontsize=12, handlelength=0, handletextpad=0,
#                  framealpha=0.7)
    
#     # Formatting
#     plt.xlabel('Time to Maturity (Years)', fontsize=14, labelpad=10)
#     plt.ylabel('Yield (%)', fontsize=14, labelpad=10)
#     plt.title(title or 'Yield Curve Comparison', fontsize=16, pad=15)
    
#     # Dynamic axis limits
#     y_values = []
#     if yields is not None:
#         y_values.extend(yields)
#     if fitted is not None:
#         y_values.extend(fitted)
#     if forecast is not None:
#         if isinstance(forecast, dict):
#             y_values.extend(forecast['mean'])
#             if 'lower' in forecast:
#                 y_values.extend(forecast['lower'])
#             if 'upper' in forecast:
#                 y_values.extend(forecast['upper'])
#         else:
#             y_values.extend(forecast)
    
#     if y_values:
#         y_min = min(y_values)
#         y_max = max(y_values)
#         plt.ylim(max(0, y_min-0.5), y_max + 0.5)
    
#     # Grid and spines
#     plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
#     for spine in plt.gca().spines.values():
#         spine.set_visible(False)
    
#     # Ticks
#     plt.xticks(fontsize=12)
#     plt.yticks(fontsize=12)
#     plt.minorticks_on()
#     plt.tick_params(axis='both', which='minor', length=3)
    
#     plt.tight_layout()
#     plt.show()

In [255]:
# # For fitting plot on Jan 31, 2025 data
# plot_yield_comparison(
#     times=times_jan31,
#     yields=yields_jan31,
#     fitted=fitted_yields_jan31,
#     metrics_text=metrics_text,
#     title='Model Fit: Observed vs Fitted Yield Curve'
# )

In [256]:
# # Load Feb 4 data
# times_feb04, yields_feb04 = load_yield_data(
#     '/Users/dr/Documents/GitHub/FixedIncome/STRIPS_data.csv', 
#     '2025-02-04'
# )
# observed_yields_feb4 = yields_feb04


In [257]:
observed_yields_feb4

array([3.694, 4.088, 4.169, 4.157, 4.289, 4.354, 4.403, 4.431, 4.464,
       4.495, 4.658, 2.52 , 4.081, 3.973, 4.252, 4.173, 4.357, 4.419,
       4.448, 4.483, 4.371, 4.451, 4.548, 4.597, 4.204, 4.221, 4.222,
       4.262, 4.299, 4.317, 4.387, 4.419, 4.458, 4.487, 4.486, 4.544,
       4.645, 4.243, 3.887, 4.182, 4.277, 4.345, 4.417, 4.409, 4.464,
       4.499])

In [258]:
# # Forecasting function
# def generate_forecast(model, maturities, T=4/365, dt=1/252, n_paths=5000, confidence=0.8):
#     """Generate forecast with fixed key names for confidence intervals"""
#     times_sim, r_paths = model.simulate_paths(T, dt, n_paths)
#     forecast = np.array([[model.yield_curve(t, r_paths[i,-1])*100 for t in maturities] 
#                        for i in range(n_paths)])
    
#     lower_pct = int((1-confidence)*50)
#     upper_pct = int(100 - (1-confidence)*50)
    
#     return {
#         'mean': np.mean(forecast, axis=0),
#         'median': np.median(forecast, axis=0),
#         'lower': np.percentile(forecast, lower_pct, axis=0),  # Fixed key
#         'upper': np.percentile(forecast, upper_pct, axis=0),  # Fixed key
#         'std': np.std(forecast, axis=0),
#         'confidence': confidence,
#         'paths': forecast
#     }


In [259]:
from scipy.optimize import minimize

In [260]:
# # Forecasting and optimization function
# def forecast_and_optimize(model, observed_yields, maturities, T=4/365, dt=1/252, n_paths=5000):
#     """Optimize model parameters to match observed yields"""
    
#     def objective(params):
#         """Objective function for optimization"""
#         model.update_params(params)
#         forecast = generate_forecast(model, maturities, T, dt, 1000)  # Fewer paths for speed
        
#         errors = forecast['mean'] - observed_yields
#         mse = np.mean(errors**2)
#         shape_error = np.mean((np.diff(forecast['mean']) - np.diff(observed_yields))**2)
#         reg_penalty = 0.001 * np.sum(params**2)
        
#         return mse + 0.3*shape_error + reg_penalty
    
#     # Parameter bounds
#     bounds = [
#         (0.005, 0.03), (0.005, 0.03),       # x1_0, x2_0
#         (0.01, 0.04), (0.01, 0.04),         # mu1, mu2
#         (0.0001, 0.02), (0.0001, 0.02),     # vol1, vol2
#         (0.05, 1.5), (0.01, 0.5)            # kappa1, kappa2
#     ]
    
#     initial_params = [
#         model.x1_0, model.x2_0, 
#         model.mu1, model.mu2,
#         model.vol1, model.vol2,
#         model.kappa1, model.kappa2
#     ]
    
#     result = minimize(
#         objective,
#         initial_params,
#         method='L-BFGS-B',
#         bounds=bounds,
#         options={
#             'maxiter': 50,
#             'ftol': 1e-6,
#             'gtol': 1e-5,
#             'disp': True
#         }
#     )
    
#     model.update_params(result.x)
#     final_forecast = generate_forecast(model, maturities, T, dt, n_paths)
    
#     # Calculate metrics
#     errors = final_forecast['mean'] - observed_yields
#     metrics = {
#         'RMSE': np.sqrt(np.mean(errors**2)),
#         'MAE': np.mean(np.abs(errors)),
#         'Max Error': np.max(np.abs(errors)),
#         'params': dict(zip(
#             ['x1_0', 'x2_0', 'mu1', 'mu2', 'vol1', 'vol2', 'kappa1', 'kappa2'],
#             result.x
#         ))
#     }
    
#     return metrics, final_forecast


In [261]:
# --- Forecasting and Optimization ---
def forecast_and_optimize(model, times, initial_params, bounds, forecast_date, actual_yields, dt=1/252, n_paths=1000):
    """Forecast yields for a future date and optimize parameters."""
    forecast_days = (pd.to_datetime(forecast_date) - pd.to_datetime('2025-01-31')).days
    T = forecast_days / 365.0
    
    def objective(params):
        model.update_params(params)
        _, r_paths = model.simulate_paths(T, dt, n_paths)
        r_forecast = r_paths[:, -1]
        forecast_yields = np.array([np.mean([model.yield_curve(t, r) for r in r_forecast]) for t in times])
        return np.sum((forecast_yields - actual_yields / 100) ** 2)
    
    result = least_squares(objective, initial_params, bounds=bounds, method='trf')
    model.update_params(result.x)
    
    _, r_paths = model.simulate_paths(T, dt, n_paths)
    r_forecast = r_paths[:, -1]
    forecast_yields = np.array([np.mean([model.yield_curve(t, r) for r in r_forecast]) for t in times])
    forecast_std = np.array([np.std([model.yield_curve(t, r) for r in r_forecast]) for t in times])
    
    return {
        'mean': forecast_yields * 100,
        'lower': (forecast_yields - 1.96 * forecast_std) * 100,
        'upper': (forecast_yields + 1.96 * forecast_std) * 100,
        'confidence': 0.95,
        'params': result.x
    }

def forecast_yields(model, times, forecast_date, dt=1/252, n_paths=1000):
    """Forecast yields for a future date without optimization."""
    forecast_days = (pd.to_datetime(forecast_date) - pd.to_datetime('2025-01-31')).days
    T = forecast_days / 365.0
    
    _, r_paths = model.simulate_paths(T, dt, n_paths)
    r_forecast = r_paths[:, -1]
    forecast_yields = np.array([np.mean([model.yield_curve(t, r) for r in r_forecast]) for t in times])
    forecast_std = np.array([np.std([model.yield_curve(t, r) for r in r_forecast]) for t in times])
    
    return {
        'mean': forecast_yields * 100,
        'lower': (forecast_yields - 1.96 * forecast_std) * 100,
        'upper': (forecast_yields + 1.96 * forecast_std) * 100,
        'confidence': 0.95
    }

In [262]:
# --- Metrics Calculation ---
def calculate_metrics(observed, predicted):
    """Calculate RMSE and R-squared for observed vs predicted yields."""
    rmse = np.sqrt(np.mean((observed - predicted) ** 2))
    y_mean = np.mean(observed)
    ss_tot = np.sum((observed - y_mean) ** 2)
    ss_res = np.sum((observed - predicted) ** 2)
    r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else np.nan
    return rmse, r2

In [263]:
# Load data
# times_feb04, yields_feb04 = load_yield_data('STRIPS_data.csv', '2025-02-04')

# print(f"Matruities_Feb, {times_feb04}")
# print(f"Yields_Feb, {yields_feb04}")

In [264]:
# Run optimization
# metrics, forecast = forecast_and_optimize(
#     model, 
#     yields_feb04, 
#     times_feb04
# )

In [265]:
# plot_yield_comparison(
#     times=times_feb04,
#     yields=yields_feb04,  
#     forecast=forecast,
#     # historical=[
#     #     {'times': times_jan31, 'yields': yields_jan31,
#     #      'label': f'Jan 31 (Short: {yields_jan31[0]:.2f}%)',
#     #      'color': '#e74c3c', 'marker': 'o'},
#     # ],
#     title='Yield Curve Forecast Comparison_Forecast and Optimze Feb 4, 2025',
# )

In [266]:
# # Load the CSV for March 4th and April 4th STRIPS data
# df = pd.read_csv('/Users/dr/Documents/GitHub/FixedIncome/STRIPS_data_Mar_Apr_2025.csv')

# # Check column names
# print("Column names:", df.columns.tolist())

# # Check unique dates in the DataFrame
# print("Unique dates in the CSV:", df['Date'].unique())

# # Load actual data for March 4, 2025, and April 4, 2025
# times_mar04, yields_mar04 = load_yield_data(
#     '/Users/dr/Documents/GitHub/FixedIncome/STRIPS_data_Mar_Apr_2025.csv',
#     '03/04/2025'  # Match the CSV date format MM/DD/YYYY
# )
# times_apr04, yields_apr04 = load_yield_data(
#     '/Users/dr/Documents/GitHub/FixedIncome/STRIPS_data_Mar_Apr_2025.csv',
#     '04/04/2025'  # Match the CSV date format MM/DD/YYYY
# )

# yields_mar04 = yields_mar04 / 100  # Convert to decimal
# yields_apr04 = yields_apr04 / 100  # Convert to decimal

In [267]:
# # Run optimization
# mar_metrics, mar_forecast = forecast_and_optimize(
#     model, 
#     yields_mar04, 
#     times_mar04
# )

In [268]:
# plot_yield_comparison(
#     times=times_mar04,
#     yields=yields_mar04,  
#     forecast=mar_forecast,
#     historical=[
#         # {'times': times_jan31, 'yields': yields_jan31,
#         #  'label': f'Jan 31 (Short: {yields_jan31[0]:.2f}%)',
#         #  'color': '#e74c3c', 'marker': 'o'},
#         # {'times': times_feb04, 'yields': yields_feb04,
#         #  'label': f'Feb 4 (Short: {yields_feb04[0]:.2f}%)',
#         #  'color': '#2ecc71', 'marker': 'o'},
#         # {'times': times_feb04, 'yields': forecast['mean'],
#         #  'label': f'Feb 4_Forecast (Short: {yields_feb04[0]:.2f}%)',
#         #  'color': '#2ecc71', 'marker': 'o'},
#         {'times': times_mar04, 'yields': yields_mar04,
#          'label': f'Mar 4 (Short: {yields_mar04[0]:.2f}%)',
#          'color': '#000000', 'marker':'o'},
#     ],
#     title='Yield Curve Forecast Comparison_Forecast and Optimze Feb 4, 2025',
# )

In [269]:
# # # Forecast yields

# def forecast_yields(model, times_jan31, yields_jan31, observed_yields_feb4,
#                    forecast_maturities=None, T=32/365, dt=1/252, n_paths=5000):
#     """Forecast yields using consistent key names"""
#     if forecast_maturities is None:
#         forecast_maturities = np.linspace(0.1, 15, 40)
    
#     forecast_stats = generate_forecast(model, forecast_maturities, T, dt, n_paths)
    
#     forecast_df = pd.DataFrame({
#         'Maturity': forecast_maturities,
#         'Mean_Yield': forecast_stats['mean'],
#         'Median_Yield': forecast_stats['median'],
#         f'Lower_{int((1-forecast_stats["confidence"])*50)}th': forecast_stats['lower'],
#         f'Upper_{int(100-(1-forecast_stats["confidence"])*50)}th': forecast_stats['upper'],
#         'Std_Dev': forecast_stats['std']
#     })
    
#     return forecast_stats, forecast_df

In [270]:
# # Generate March 4 forecast
# forecast_stats_mar4, forecast_df_mar4 = forecast_yields(
#         model, 
#         times_jan31, 
#         yields_jan31, 
#         observed_yields_feb4,
#         T=32/365  # 32 days from Jan 31 to Mar 4
#     )

In [271]:
# forecast_stats_mar4

In [272]:
# # Plot forecast for March 4, 2025
# plot_yield_comparison(
#     times=forecast_df_mar4['Maturity'],
#     forecast={
#         'mean': forecast_stats_mar4['mean'],
#         'lower': forecast_stats_mar4['lower'],
#         'upper': forecast_stats_mar4['upper'],
#         'confidence': forecast_stats_mar4['confidence']
#     },
#     historical=[
#         {'times': times_jan31, 'yields': yields_jan31,
#          'label': f'Jan 31 (Short: {yields_jan31[0]:.2f}%)',
#          'color': '#e74c3c', 'marker': 'o'},
#         {'times': times_feb04, 'yields': yields_feb04,
#          'label': f'Feb 4 (Short: {yields_feb04[0]:.2f}%)',
#          'color': '#2ecc71', 'marker': 'o'},
#         {'times': times_mar04, 'yields': yields_mar04,
#          'label': f'Mar 4 (Short: {yields_mar04[0]:.2f}%)',
#          'color': '#000000', 'marker':'o'},
#     ],
#     title='Yield Curve Forecast Comparison_March 04, 2025 Forecast'
# )

In [273]:
# Save results
# forecast_df_mar4.to_csv('vasicek_forecast_march4.csv', index=False)
# print("March 4 forecast saved to CSV")

In [274]:
# # Generate April 4 forecast
# forecast_stats_apr4, forecast_df_apr4 = forecast_yields(
#     model, 
#     times_jan31, 
#     yields_jan31, 
#     observed_yields_feb4,
#     T=64/365  # 64 days from Jan 31 to Apr 4
#     )

In [275]:
# forecast_df_apr4

In [276]:
# # Plot forecast for April 4, 2025

# plot_yield_comparison(
#     times=forecast_df_apr4['Maturity'],
#     forecast={
#         'mean': forecast_stats_apr4['mean'],
#         'lower': forecast_stats_apr4['lower'],
#         'upper': forecast_stats_apr4['upper'],
#         'confidence': forecast_stats_apr4['confidence']
#     },
#     historical=[
#         {'times': times_jan31, 'yields': yields_jan31,
#          'label': f'Jan 31 (Short: {yields_jan31[0]:.2f}%)',
#          'color': '#e74c3c', 'marker': 'o'},
#         {'times': times_feb04, 'yields': yields_feb04,
#          'label': f'Feb 4 (Short: {yields_feb04[0]:.2f}%)',
#          'color': '#2ecc71', 'marker': 'o'},
#         {'times': times_mar04, 'yields': yields_mar04,
#          'label': f'Mar 4 (Short: {yields_mar04[0]:.2f}%)',
#          'color': '#000000', 'marker':'o'},
#         {'times': times_apr04, 'yields': yields_apr04,
#          'label': f'Apr 4 (Short: {yields_apr04[0]:.2f}%)',
#          'color': '#9b59b6', 'marker':'o'}
#     ],
#     title='Yield Curve Forecast Comparison_March 04, 2025 Forecast'
# )

In [277]:
# --- Plotting ---
def plot_yield_comparison(times, yields=None, fitted=None, forecast=None, historical=None,
                          metrics_text=None, title=None, figsize=(12, 8), dpi=100):
    plt.figure(figsize=figsize, dpi=dpi)
    sorted_idx = np.argsort(times)
    times_sorted = times[sorted_idx]
    
    if yields is not None:
        plt.plot(times_sorted, yields[sorted_idx], 'o-', label='Observed Yields', color='#1f77b4', linewidth=2)
    if fitted is not None:
        plt.plot(times_sorted, fitted[sorted_idx], 's--', label='Fitted Yields', color='#d62728', linewidth=2)
    if forecast is not None:
        fcst_mean = forecast['mean'][sorted_idx]
        plt.plot(times_sorted, fcst_mean, 'D-', label='Forecasted Yields', color='#2ca02c', linewidth=2)
        if 'lower' in forecast and 'upper' in forecast:
            plt.fill_between(times_sorted, forecast['lower'][sorted_idx], forecast['upper'][sorted_idx],
                            color='#2ca02c', alpha=0.15, label=f"{forecast['confidence']*100}% Confidence Band")
    if historical is not None:
        for series in historical:
            hist_times = series['times'][np.argsort(series['times'])]
            hist_yields = series['yields'][np.argsort(series['times'])]
            plt.plot(hist_times, hist_yields, series.get('marker', 'o'), label=series.get('label'), 
                     color=series.get('color', '#7f7f7f'), markersize=8, linestyle='none')
    
    plt.xlabel('Maturity (Years)')
    plt.ylabel('Yield (%)')
    plt.title(title or 'Yield Curve Comparison')
    plt.legend(loc='best')
    if metrics_text:
        plt.text(0.05, 0.95, metrics_text, transform=plt.gca().transAxes, fontsize=10, 
                 verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    plt.grid(True)
    plt.show()

In [279]:
# --- Main Execution ---
filepath = '/Users/dr/Documents/GitHub/FixedIncome/STRIPS_data.csv'
dates = ['2025-01-31', '2025-02-04', '2025-03-04', '2025-04-04']
data = {date: load_yield_data(filepath, date) for date in dates}
times_jan31, yields_jan31 = data['2025-01-31']
times_feb04, yields_feb04 = data['2025-02-04']
times_mar04, yields_mar04 = data['2025-03-04']
times_apr04, yields_apr04 = data['2025-04-04']


Exception: Error loading data for 2025-03-04: index 0 is out of bounds for axis 0 with size 0

In [ ]:

# Initial parameters and bounds
x0 = [0.02, 0.02, 0.025, 0.025, 0.01, 0.01, 0.5, 0.1]
bounds = (
    [0.01, 0.01, 0.015, 0.015, 0.001, 0.001, 0.05, 0.01],
    [0.03, 0.03, 0.04, 0.04, 0.02, 0.02, 1.0, 0.3]
)


In [ ]:

# Fit Jan 31 data
model = Two_Factor_Vasicek(*x0)
fitter = Fitter(model, yields_jan31 / 100, times_jan31)
fit_result = fitter.fit(x0, solver='trf', bounds=bounds)
fitted_yields = np.array([model.yield_curve(t, model.x1_0 + model.x2_0) for t in times_jan31]) * 100
r2_jan31 = fitter.r_squared()
rmse_jan31 = fitter.rmse() * 100
metrics_text_jan31 = f'$R^2$: {r2_jan31:.2f}\nRMSE: {rmse_jan31:.2f}%'


In [ ]:

# Plot Jan 31 fit
plot_yield_comparison(
    times=times_jan31,
    yields=yields_jan31,
    fitted=fitted_yields,
    metrics_text=metrics_text_jan31,
    title='Jan 31, 2025 Yield Curve Fit'
)


In [ ]:

# Forecast and optimize for Feb 4
feb04_forecast = forecast_and_optimize(model, times_feb04, fit_result.x, bounds, '2025-02-04', yields_feb04)
rmse_feb04, r2_feb04 = calculate_metrics(yields_feb04, feb04_forecast['mean'])
metrics_text_feb04 = f'$R^2$: {r2_feb04:.2f}\nRMSE: {rmse_feb04:.2f}%'


In [ ]:

# Plot Feb 4 forecast
plot_yield_comparison(
    times=times_feb04,
    yields=yields_feb04,
    forecast=feb04_forecast,
    historical=[
        {'times': times_jan31, 'yields': yields_jan31, 'label': f'Jan 31 (Short: {yields_jan31[0]:.2f}%)', 'color': '#e74c3c', 'marker': 'o'},
        {'times': times_feb04, 'yields': yields_feb04, 'label': f'Feb 4 (Short: {yields_feb04[0]:.2f}%)', 'color': '#2ecc71', 'marker': 'o'}
    ],
    metrics_text=metrics_text_feb04,
    title='Yield Curve Forecast for Feb 4, 2025'
)


In [ ]:

# Forecast and optimize for Mar 4
mar04_forecast = forecast_and_optimize(model, times_mar04, feb04_forecast['params'], bounds, '2025-03-04', yields_mar04)
rmse_mar04, r2_mar04 = calculate_metrics(yields_mar04, mar04_forecast['mean'])
metrics_text_mar04 = f'$R^2$: {r2_mar04:.2f}\nRMSE: {rmse_mar04:.2f}%'


In [ ]:

# Plot Mar 4 forecast
plot_yield_comparison(
    times=times_mar04,
    yields=yields_mar04,
    forecast=mar04_forecast,
    historical=[
        {'times': times_jan31, 'yields': yields_jan31, 'label': f'Jan 31 (Short: {yields_jan31[0]:.2f}%)', 'color': '#e74c3c', 'marker': 'o'},
        {'times': times_feb04, 'yields': yields_feb04, 'label': f'Feb 4 (Short: {yields_feb04[0]:.2f}%)', 'color': '#2ecc71', 'marker': 'o'},
        {'times': times_mar04, 'yields': yields_mar04, 'label': f'Mar 4 (Short: {yields_mar04[0]:.2f}%)', 'color': '#000000', 'marker': 'o'}
    ],
    metrics_text=metrics_text_mar04,
    title='Yield Curve Forecast for Mar 4, 2025'
)


In [ ]:

# Forecast for Apr 4 (no optimization)
apr04_forecast = forecast_yields(model, times_apr04, '2025-04-04')
rmse_apr04, r2_apr04 = calculate_metrics(yields_apr04, apr04_forecast['mean'])
metrics_text_apr04 = f'$R^2$: {r2_apr04:.2f}\nRMSE: {rmse_apr04:.2f}%'


In [ ]:

# Plot Apr 4 forecast with historical data
plot_yield_comparison(
    times=times_apr04,
    yields=yields_apr04,
    forecast=apr04_forecast,
    historical=[
        {'times': times_jan31, 'yields': yields_jan31, 'label': f'Jan 31 (Short: {yields_jan31[0]:.2f}%)', 'color': '#e74c3c', 'marker': 'o'},
        {'times': times_feb04, 'yields': yields_feb04, 'label': f'Feb 4 (Short: {yields_feb04[0]:.2f}%)', 'color': '#2ecc71', 'marker': 'o'},
        {'times': times_mar04, 'yields': yields_mar04, 'label': f'Mar 4 (Short: {yields_mar04[0]:.2f}%)', 'color': '#000000', 'marker': 'o'},
        {'times': times_apr04, 'yields': yields_apr04, 'label': f'Apr 4 (Short: {yields_apr04[0]:.2f}%)', 'color': '#9b59b6', 'marker': 'o'}
    ],
    metrics_text=metrics_text_apr04,
    title='Yield Curve Forecast for April 4, 2025'
)


In [ ]:

# Print metrics
print("Jan 31 Fit Metrics:")
print(metrics_text_jan31)
print("\nFeb 4 Forecast Metrics:")
print(metrics_text_feb04)
print("\nMar 4 Forecast Metrics:")
print(metrics_text_mar04)
print("\nApr 4 Forecast Metrics:")
print(metrics_text_apr04)